# Notebook to design trading bot

Latest version: 2024-08-14  
Author: MvS

## Description

Example of trend-following trading bot using SMA crossovers as signals

## Result




In [ ]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from lumibot.entities import TradingFee, Asset

from alpaca_trade_api import REST
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from dotenv import dotenv_values
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s")

# from utils.finbert_utils import estimate_sentiment

env_dict = dotenv_values("../.env")

In [ ]:
ALPACA_CREDS = {
    "BASE_URL": env_dict['ALP_API_URL'],
    "API_KEY": env_dict['ALP_API_KEY'],
    "API_SECRET": env_dict['ALP_API_SECRET'],
    "PAPER": True,
}

ALPACA_CREDS

In [ ]:
class TrendFollowing(Strategy):
    def initialize(self):
        self.loss_ratio = self.parameters["loss_ratio"]
        self.profit_ratio = self.parameters["profit_ratio"]

        assert len(self.parameters["sma_periods"]) == 2
        assert min(self.parameters["sma_periods"]) < max(self.parameters["sma_periods"])

        self.min_sma = min(self.parameters["sma_periods"])
        self.max_sma = max(self.parameters["sma_periods"])

        self.cash_at_risk = self.parameters["cash_at_risk"]
        self.symbol = self.parameters["symbol"]
        self.sleeptime = self.parameters["sleeptime"]

        self.mode = None
        self.signal = None
        self.start = "2023-01-01"

    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)

        return cash, last_price, quantity

    def on_trading_iteration(self):
        bars = self.get_historical_prices(self.symbol, self.max_sma + 1, "day")
        sym_df = bars.df

        # sym_df = pd.DataFrame(yf.download(self.symbol, self.start)['Close'])
        sym_df['short-per'] = sym_df['close'].rolling(self.min_sma).mean()
        sym_df['long-per'] = sym_df['close'].rolling(self.max_sma).mean()
        sym_df['Signal'] = np.where(
            np.logical_and(
                sym_df['short-per'] > sym_df['long-per'],
                sym_df['short-per'].shift(1) < sym_df['long-per'].shift(1),
            ),
            "BUY",
            None,
        )
        sym_df['Signal'] = np.where(
            np.logical_and(
                sym_df['short-per'] < sym_df['long-per'],
                sym_df['short-per'].shift(1) > sym_df['long-per'].shift(1),
            ),
            "SELL",
            sym_df['Signal'],
        )
        sym_df['Mode'] = np.where(
            sym_df['short-per'] > sym_df['long-per'], "LONG", "SHORT"
        )
        self.signal = sym_df.iloc[-1].Signal
        self.mode = sym_df.iloc[-1].Mode

        pos = self.get_position(self.symbol)
        quantity = 1

        if self.signal == 'BUY' or (self.mode == 'LONG' and pos is None):
            logging.info(pos)
            if pos is not None:
                self.sell_all()

            cash, last_price, quantity = self.position_sizing()
            order = self.create_order(
                asset=self.symbol,
                quantity=quantity,
                side="buy",
                type="bracket",
                take_profit_price=last_price * self.profit_ratio,
                stop_loss_price=last_price * self.loss_ratio,
                stop_loss_limit_price=last_price * self.loss_ratio - 1.0,
            )
            self.submit_order(order)

        elif self.signal == 'SELL':
            if pos is not None:
                self.sell_all()

            # cash, last_price, quantity = self.position_sizing()
            # order = self.create_order(
            #     asset=self.symbol,
            #     quantity=quantity,
            #     side="sell",
            #     type="bracket",
            #     take_profit_price=last_price * self.profit_ratio,
            #     stop_loss_price=last_price * self.loss_ratio,
            #     stop_loss_limit_price=last_price * self.loss_ratio + 1.0,
            # )
            # self.submit_order(order)

In [ ]:
trading_symbol = "AAPL"
benchmark_symbol = "AAPL"

# Set strategy parameters
parameters = {
    "symbol": trading_symbol,
    "cash_at_risk": 0.5,
    "sleeptime": '1D',
    "sma_periods": [9, 21],
    "profit_ratio": 1.10,
    "loss_ratio": 0.96,
    "benchmark_asset": Asset(symbol=benchmark_symbol, asset_type="stock"),
    "credentials": ALPACA_CREDS,
}

trade = False
if trade:
    broker = Alpaca(ALPACA_CREDS)  # Initialize Alpaca broker with credentials

    strategy = TrendFollowing(
        name='Trendbot',
        broker=broker,
        parameters=parameters,
        force_start_immediately=True,
    )

    bot = Trader()
    bot.add_strategy(strategy)
    bot.run_all()
else:
    # Set a flat $5 trading fee for each trade
    trading_fee = TradingFee(flat_fee=5)

    start_date = datetime(2023, 1, 1)
    end_date = datetime.today() - timedelta(days=30)
    TrendFollowing.backtest(
        YahooDataBacktesting,
        start_date,
        end_date,
        benchmark_asset=benchmark_symbol,
        budget=10000,
        buy_trading_fees=[trading_fee],
        sell_trading_fees=[trading_fee],
        parameters=parameters,
    )

## Testing area for indicator analytics

### Loading data

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from importlib import reload  # Python 3.4+
import utils.indicator_utils as idc

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s")

symbol = 'AAPL'
start = "2023-01-01"
logging.info(f"getting data from Yahoo Finance for symbol: {symbol}")

sym_df = pd.DataFrame(yf.download(symbol, start, progress=False))
logging.info(f"data frame contains {sym_df.shape} points")
sym_df.tail(5)

### Generate indicators

In [ ]:
reload(idc)

min_ma = 9
max_ma = 26

sym_df['short-sma'] = idc.sma(sym_df['Close'], length=min_ma)
sym_df['long-sma'] = idc.sma(sym_df['Close'], length=max_ma)
sym_df['short-ema'] = idc.ema(sym_df['Close'], length=min_ma)
sym_df['long-ema'] = idc.ema(sym_df['Close'], length=max_ma)

sym_df['atr'] = idc.vola_range(close=sym_df['Close'], high=sym_df['High'], low=sym_df['Low'], mode='ATR', length=min_ma, percentile=True)
sym_df['ltr'] = idc.vola_range(close=sym_df['Close'], high=sym_df['High'], low=sym_df['Low'], mode='ALR', length=1)

# Verify beginning of table has no gaps
sym_df.head(5)

### Calculate simple trading signals

In [ ]:
reload(idc)

# Clean up
sym_df.drop(
    ['Mode', 'P_Signal', 'N_Signal', 'Buy', 'Sell',],
    axis=1,
    inplace=True,
    errors='ignore',
)

# Generate crossover signals
temp = idc.crossovers(short=sym_df['short-sma'], long=sym_df['long-sma'], scaledsignal=False)
sym_df = pd.concat([sym_df, temp], axis=1)

# Re-label columns
sym_df = sym_df.rename(columns={'P_Signal': 'Buy', 'N_Signal': 'Sell'})
del temp

### Plot results

In [ ]:
# Define the columns and their styles
columns = [
    'short-sma',
    'long-sma',
    'Mode',
    'Buy',
    'Sell',
]
colors = ['darkgreen', 'orangered', 'black', 'darkgreen', 'orangered']
linestyles = ['-', '-', ':', None, None]
markers = [None, None, None, '^', 'v']


# Plot each column separately
ax = None
for col, color, style, marker in zip(columns, colors, linestyles, markers):
    ax = sym_df[col].plot(color=color, linestyle=style, marker=marker, ax=ax)
ax.legend(loc='upper left')